In [1]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn

from mindnlp import load_dataset

from mindnlp.engine import Trainer, Evaluator
from mindnlp.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp.metrics import Accuracy

/opt/conda/envs/llm/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
imdb_ds = load_dataset('imdb', split=['train', 'test'],verification_mode='no_checks')
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']

HF google storage unreachable. Downloading and preparing it from source
Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 86.42it/s]


Dataset parquet downloaded and prepared to /code/.mindnlp/datasets/imdb/parquet/plain_text-98e5755e821b676c/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 286.02it/s]


In [10]:
imdb_train.get_dataset_size()

25000

In [11]:
import numpy as np
#处理数据集
#将原始文本数据集转换为适用于NLP模型训练的格式
def process_dataset(dataset, tokenizer, max_seq_len=512, batch_size=16, shuffle=False):
    def tokenize(text):
        tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized[0], tokenized[1]

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})

    return dataset

In [12]:
from mindnlp.transformers import GPTTokenizer
# tokenizer
#初始化 GPTTokenizer，并向其添加一些特殊标记
gpt_tokenizer = GPTTokenizer.from_pretrained('openai-gpt', from_pt=True)

# add sepcial token: <PAD>
special_tokens_dict = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
}
num_added_toks = gpt_tokenizer.add_special_tokens(special_tokens_dict)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [13]:
# split train dataset into train and valid datasets
imdb_train, imdb_val = imdb_train.split([0.7, 0.3])

In [14]:
dataset_train = process_dataset(imdb_train, gpt_tokenizer, shuffle=True)
dataset_val = process_dataset(imdb_val, gpt_tokenizer)
dataset_test = process_dataset(imdb_test, gpt_tokenizer)

In [16]:
next(dataset_train.create_tuple_iterator())

[Tensor(shape=[16, 512], dtype=Int64, value=
 [[ 1393,  1133,   240 ... 40480, 40480, 40480],
  [ 4868,   278,  1558 ... 40480, 40480, 40480],
  [10068, 21377,   488 ... 40480, 40480, 40480],
  ...
  [  249,   668,   939 ... 40480, 40480, 40480],
  [  655,   641,   246 ... 40480, 40480, 40480],
  [  616,   544,  2263 ... 40480, 40480, 40480]]),
 Tensor(shape=[16, 512], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[16], dtype=Int32, value= [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1])]

In [18]:
from mindnlp.transformers import GPTForSequenceClassification
#使用 MindNLP 库中的 GPTForSequenceClassification 模型进行微调
#微调 GPT 模型以执行文本分类任务，并配置了相应的训练设置和回调函数。

# set bert config and define parameters for training
model = GPTForSequenceClassification.from_pretrained('openai-gpt', from_pt=True, num_labels=2)
model.config.pad_token_id = gpt_tokenizer.pad_token_id
model.resize_token_embeddings(model.config.vocab_size + 3)

optimizer = nn.Adam(model.trainable_params(), learning_rate=2e-5)

metric = Accuracy()

#定义回调函数
# define callbacks to save checkpoints
ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='gpt_imdb_finetune', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='gpt_imdb_finetune_best', auto_load=True)

trainer = Trainer(network=model, train_dataset=dataset_train,
                  eval_dataset=dataset_train, metrics=metric,
                  epochs=3, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb],
                  jit=False)

100%|██████████| 457M/457M [01:19<00:00, 6.03MB/s] 
The following parameters in models are missing parameter:
['score.weight']


In [19]:
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0: 100%|██████████| 1094/1094 [14:41<00:00,  1.24it/s, loss=0.24903066]


Checkpoint: 'gpt_imdb_finetune_epoch_0.ckpt' has been saved in epoch: 0.


Evaluate: 100%|██████████| 1094/1094 [07:45<00:00,  2.35it/s]


Evaluate Score: {'Accuracy': 0.9527428571428571}
---------------Best Model: 'gpt_imdb_finetune_best.ckpt' has been saved in epoch: 0.---------------


...

Evaluate Score: {'Accuracy': 0.9793714285714286}
---------------Best Model: 'gpt_imdb_finetune_best.ckpt' has been saved in epoch: 1.---------------


Epoch 2: 100%|██████████| 1094/1094 [14:27<00:00,  1.26it/s, loss=0.09556616] 


The maximum number of stored checkpoints has been reached.
Checkpoint: 'gpt_imdb_finetune_epoch_2.ckpt' has been saved in epoch: 2.


Evaluate: 100%|██████████| 1094/1094 [08:20<00:00,  2.18it/s]


Evaluate Score: {'Accuracy': 0.9686285714285714}
Loading best model from 'checkpoint' with '['Accuracy']': [0.9793714285714286]...
---------------The model is already load the best model from 'gpt_imdb_finetune_best.ckpt'.---------------


In [20]:
evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
evaluator.run(tgt_columns="labels")

Evaluate: 100%|██████████| 1563/1563 [10:59<00:00,  2.37it/s]

Evaluate Score: {'Accuracy': 0.93484}
